###### Nipype Workflow for ProbTrackX2 fdt_path waypath thresholding and normalization
## Version 9: Threshold then normalize

In [2]:
from nipype import config
cfg = dict(execution={'remove_unnecessary_outputs': False,
                      'keep_inputs': True},
           monitoring={'enabled': True,
                       'sample_frequency': 5,
                       'summary_file': './resource_monitor.json'
                      }
          )
config.update_config(cfg)

In [3]:
from os.path import abspath
from IPython.display import Image

from nipype import SelectFiles, Node, MapNode, Workflow, Function
from nipype.interfaces import fsl
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import ImageStats
from nipype.interfaces.fsl import Merge
from nipype.interfaces.io import DataSink
from nipype.interfaces.utility import IdentityInterface

import networkx
import graphviz as gv
import os
import base64
import re
import glob
import sys
import subprocess

import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import copy

## Select Files:

In [14]:
# Node: SelectFiles
## Selects subject folders from "/data/NipypeScratch/runpbx2/"
#s_templates={'subject': '_subject_id_100[2-3]*'} # 2 subjects
s_templates={ 'fdt_paths': '_subject_id_*/_seed*/pbx2/fdt_paths.nii.gz' } 

sf0 = Node(SelectFiles(s_templates),
          name='selectfiles0'
          )
sf0.inputs.base_directory = '/data/NipypeScratch/runpbx2/'

# Save fdt_paths list as variable 'fdt+paths'
sf0_results= sf0.run().outputs
fdt_paths= sf0_results.fdt_paths

180731-16:59:29,587 workflow INFO:
	 [Node] Setting-up "selectfiles0" in "/tmp/tmpsnnfwri3/selectfiles0".
180731-16:59:29,592 workflow INFO:
	 [Node] Running "selectfiles0" ("nipype.interfaces.io.SelectFiles")
180731-16:59:31,558 workflow INFO:
	 [Node] Finished "selectfiles0".


In [15]:
infosource = Node(IdentityInterface(fields=['fdt_paths']),
                  name="infosource")
infosource.iterables = [ ('fdt_paths', fdt_paths) ]

## Normalize fdt_paths using waytotal:

In [16]:
# Node: Function: Normalizes using FSL Maths -div
# ifiles: original input file
Normalize = Node(fsl.BinaryMaths(operation='div'),
                 name = 'Normalize',
                )
Normalize.inputs.output_type= 'NIFTI_GZ'
# Normalize.in_file= # From pipeline: fdt_paths
# Normalize.inputs.operand_value= # From pipeline: waytotal

## Threshold fdt_paths using waytotal, Should not account for waytotal if already normalized:

In [17]:
# Node: Function: Thresholds using FSL Maths
# ifiles: original input file
Threshold = Node(fsl.Threshold(),
                 name = 'Threshold',
                )
#Threshold.inputs.thresh= 1 # Normalized threshold is 1
Threshold.output_type= 'NIFTI_GZ'
#Threshold.use_robust_range= True

#Threshold.in_file= # From pipeline: fdt_paths
#Threshold.thresh= # From pipeline: waytotal/or constant
#Threshold.out_file= # From pipeline: OPTIONAL

## Organize thresholded niftis in /SASRAID/ThreshNipype/output

In [18]:
Sink = Node(DataSink(), name = 'Sink')
Sink.inputs.base_directory= '/data/SASRAID/ThreshNipype/output'
Sink.inputs.substitutions= [('_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_', ''),
                            ('.._seed_..data..HCP_Data..EHECHT_ROIS..', ''),
                           ('.nii.gz..pbx2..fdt_paths.nii.gz','')]

## Initialize Workflows:

In [19]:
# Workflow: Initialize
wf = Workflow(name="ThreshNipype")
wf.base_dir = '/data/SASRAID/'

## Connect Nodes:

In [20]:
def getWaytotal(x, Tscaling=10000): 
    import subprocess
    import re
    
    x= re.sub('fdt_paths.nii.gz', 'waytotal', x)
    mycommand= "cat "+str(x)
    mycommand= mycommand.split(sep=' ')
    result= subprocess.run(mycommand, stdout=subprocess.PIPE)
    tmpwaytotal= result.stdout.decode("utf-8").split(' ')[0]
    #print('waytotal: %s' % tmpwaytotal)
    
    if(not tmpwaytotal): 
        print('Waytotal Empty for: %s' % pathwaytotal)
    else:
        waytotals= float(tmpwaytotal) / Tscaling                
        return( waytotals )

In [21]:
def cleanOutput(x):
    import re
    
    x= re.search('_id_(.*?).._seed_..data.*EHECHT_ROIS..(.*?)_(.*?)_(.*?).nii.gz', x)
    x= '%s_%s_%s_%s' % (x.group(1), x.group(2), x.group(3), x.group(4))
    return(x)

In [22]:
# Connect Nodes
wf.connect(infosource, "fdt_paths", Threshold, "in_file")
wf.connect(infosource, ("fdt_paths", getWaytotal, 10000), Threshold, "thresh")

wf.connect(Threshold, "out_file", Normalize, "in_file")
wf.connect(infosource, ("fdt_paths", getWaytotal, 1), Normalize, "operand_value")

# wf.connect(Threshold, ("out_file", cleanOutput), Sink, "base_directory")
# wf.connect(Threshold, "out_file", Sink, "container")
wf.connect(Normalize, 'out_file', Sink, 'fdt_normm_thresh')

## Run Workflow:

In [23]:
# Run Workflow
#wf.run()
wf.run(plugin='MultiProc', plugin_args={'n_procs': 25})

180731-17:00:00,415 workflow INFO:
	 Workflow ThreshNipype settings: ['check', 'execution', 'logging', 'monitoring']
180731-17:03:00,202 workflow INFO:
	 Running in parallel.
180731-17:03:00,728 workflow INFO:
	 [MultiProc] Running 0 tasks, and 2760 jobs ready. Free memory (GB): 169.98/169.98, Free processors: 25/25.
180731-17:03:00,905 workflow INFO:
	 [Job 9] Cached (ThreshNipype.Threshold).
180731-17:03:00,906 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Threshold" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_899885.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz..pbx2..fdt_paths.nii.gz/Threshold".180731-17:03:00,907 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Threshold" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_899885.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Left.nii.gz..pbx2..fdt_paths.nii.gz/Threshold".180731-17:03:00,908 workflow INFO:
	 [Node] Sett

fslmaths /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_899885.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Threshold/fdt_paths_thresh.nii.gz -div 3195000.00000000 /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_899885.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Normalize/fdt_paths_thresh_maths.nii.gz
180731-17:03:03,266 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_898176.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz..pbx2..fdt_paths.nii.gz/Normalize".
180731-17:03:03,268 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_898176.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Left.nii.gz..pbx2..f

	 [Node] Finished "ThreshNipype.Sink".

180731-17:03:05,497 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")180731-17:03:05,497 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896879.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz..pbx2..fdt_paths.nii.gz/Normalize".


180731-17:03:05,500 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896879.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Left.nii.gz..pbx2..fdt_paths.nii.gz/Normalize".180731-17:03:05,502 interface INFO:
	 sub: /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_898176.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz..pbx2..fdt_paths.nii.gz/fdt_paths_thresh_maths.nii.gz -> /data/SASRAID/ThreshNipype

	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")
180731-17:03:07,604 workflow INFO:
	 [Job 81] Cached (ThreshNipype.Threshold).

180731-17:03:07,596 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896778.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Left.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-17:03:07,598 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")180731-17:03:07,617 workflow INFO:
	 [Job 84] Cached (ThreshNipype.Threshold).
180731-17:03:07,599 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-17:03:07,599 interface INFO:
	 sub: /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896879.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Left.nii.gz..pbx2..fdt_paths.nii.gz/fdt_paths_thresh_maths.nii.gz -> /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/896879Human_Hypot

fslmaths /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894774.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Threshold/fdt_paths_thresh.nii.gz -div 3195000.00000000 /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894774.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Normalize/fdt_paths_thresh_maths.nii.gz
180731-17:03:10,85 workflow INFO:
	 [Job 88] Cached (ThreshNipype.Normalize).
180731-17:03:10,98 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894673.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz..pbx2..fdt_paths.nii.gz/Normalize".
180731-17:03:10,101 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Normalize" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894673.

...

Process NonDaemonPoolWorker-767:
Traceback (most recent call last):
Process NonDaemonPoolWorker-771:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process NonDaemonPoolWorker-770:
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/conda/lib/python3.6/multiprocessing/queue

KeyboardInterrupt: 

Process NonDaemonPoolWorker-587:
Process NonDaemonPoolWorker-752:
Process NonDaemonPoolWorker-781:
Process NonDaemonPoolWorker-768:
Process NonDaemonPoolWorker-777:
Process NonDaemonPoolWorker-772:
Process NonDaemonPoolWorker-614:
Process NonDaemonPoolWorker-618:
Process NonDaemonPoolWorker-748:
Process NonDaemonPoolWorker-773:
Process NonDaemonPoolWorker-779:
Process NonDaemonPoolWorker-769:
Process NonDaemonPoolWorker-526:
Process NonDaemonPoolWorker-782:
Process NonDaemonPoolWorker-527:
Process NonDaemonPoolWorker-529:
Process NonDaemonPoolWorker-489:
Process NonDaemonPoolWorker-508:
Process NonDaemonPoolWorker-774:
Process NonDaemonPoolWorker-545:
Process NonDaemonPoolWorker-534:
Process NonDaemonPoolWorker-620:
Process NonDaemonPoolWorker-538:
Process NonDaemonPoolWorker-470:
Process NonDaemonPoolWorker-775:
Process NonDaemonPoolWorker-780:
Process NonDaemonPoolWorker-500:
Process NonDaemonPoolWorker-515:
Process NonDaemonPoolWorker-778:
Process NonDaemonPoolWorker-776:
Traceback 

  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
 

  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


# Merge Males/Females

In [24]:
metaloc= '/code/notebooks/postPBX/unrestricted_dagutman_7_12_2018_15_39_53.csv'
metadata= pd.read_csv(metaloc,header=0)

In [40]:
pbx_dir= '/data/SASRAID/ThreshNipype/output/fdt_normm_thresh/*'
pbx_results= [ fn for fn in glob.glob(pbx_dir) ]

subjectinfo= {}
for i in pbx_results:

    output_sid= re.search('.*/(.*?)Human.*', i).group(1)
    fdt_paths= glob.glob( i+'/fdt_paths*.nii.gz' )
    subjectinfo[output_sid]= fdt_paths[0]

In [41]:
# Copy original metadata
finalmetadata= copy.deepcopy(metadata)

In [42]:
# Set FDT_Paths column
filler= [''] * metadata.shape[0]
for k,v in subjectinfo.items():
    k= int(k)
    idx_row= int( np.where(finalmetadata['Subject'] == k)[0][0] )
    filler[idx_row]= v
finalmetadata['FDT_Paths']= filler

In [44]:
gender_info= finalmetadata.groupby(by='Gender')

In [45]:
# Subject ID grouped by gender:
Females= list( gender_info.get_group('F')['FDT_Paths'] )
Males= list( gender_info.get_group('M')['FDT_Paths'] )

In [46]:
females= [ i for i in Females if len(i) > 0]
males= [ i for i in Males if len(i) > 0]

## Identity: Female/Male

In [48]:
infosourceG = Node(IdentityInterface(fields=['Female','Male']),
                  name="infosourceG")
infosourceG.iterables = [ ('Female', females), ('Male', males) ]

## Select Files

## Merge Niftis

In [51]:
merger = Node(Merge(),
              name = 'Merger'
             )
merger.inputs.dimension= 't'
merger.inputs.output_type= 'NIFTI_GZ'
merger.inputs.merged_file= '/data/SASRAID/ThreshNipype/GenderDiff/Females'
merger.inputs.in_files= females


merger.run().outputs

180731-17:23:15,685 workflow INFO:
	 [Node] Setting-up "Merger" in "/tmp/tmptrrncm5c/Merger".
180731-17:23:15,720 workflow INFO:
	 [Node] Running "Merger" ("nipype.interfaces.fsl.utils.Merge"), a CommandLine Interface with command:
fslmerge -t /data/SASRAID/ThreshNipype/GenderDiff/Females /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/100307Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/101006Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/101915Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/102311Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/102816Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/103414Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype

180731-17:26:04,753 workflow WARNING:
	 [Node] Error on "Merger" (/tmp/tmptrrncm5c/Merger)


TraitError: The trait 'merged_file' of a MergeOutputSpec instance is an existing file name, but the path  '/data/SASRAID/ThreshNipype/GenderDiff/Females' does not exist.

In [53]:
### NOW RUN MALES
merger.inputs.merged_file= '/data/SASRAID/ThreshNipype/GenderDiff/Males'
merger.inputs.in_files= males


merger.run().outputs

180731-17:27:35,703 workflow INFO:
	 [Node] Setting-up "Merger" in "/tmp/tmptrrncm5c/Merger".
180731-17:27:35,732 workflow INFO:
	 [Node] Running "Merger" ("nipype.interfaces.fsl.utils.Merge"), a CommandLine Interface with command:
fslmerge -t /data/SASRAID/ThreshNipype/GenderDiff/Males /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/100206Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/100408Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/100610Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/101107Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/101309Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/output/fdt_normm_thresh/101410Human_BasalForebrain_Bilat/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/o

180731-17:29:46,625 workflow WARNING:
	 [Node] Error on "Merger" (/tmp/tmptrrncm5c/Merger)


TraitError: The trait 'merged_file' of a MergeOutputSpec instance is an existing file name, but the path  '/data/SASRAID/ThreshNipype/GenderDiff/Males' does not exist.

261

In [ ]:
print("Hello")

In [ ]:
# Connect Nodes
wf.connect(males, merger, "in_files")
wf.connect(females, "Females", merger, "in_files")